In [13]:
# importujemy wymagane biblioteki
from time import sleep
from datetime import datetime

from selenium.webdriver import Edge
from selenium.webdriver.firefox.service import Service

from bs4 import BeautifulSoup

In [14]:
def download_page(page_number: int, job_name: str) -> BeautifulSoup:
    """
    Wchodzi na stronę oraz pobiera jej całkowitą zawartość
    """
    global browser
    url = 'https://nofluffjobs.com/pl/jobs/?criteria=keyword%3D"{job_name}"'
    browser.get(url.format(
        job_name=job_name,
        page_number=page_number)
        )
    cockies_btn = browser.find_elements('id', 'onetrust-accept-btn-handler')
    if len(cockies_btn) > 0 :
        cockies_btn[0].click()
    html = browser.page_source

    return BeautifulSoup(html)

In [15]:
def is_empty_page(page) -> bool:
    """
    Sprawdza czy strona jest pusta
    """
    empty_condition = page.find(id='undefinedJobs')

    if empty_condition is None:
        return True
    else:
        return False

In [16]:
def get_job_offers(job_name: str) -> list:
    """
    Pobiera aktualnie dostępne oferty pracy dla danego zapytania

    @param job_name: zapytanie, które chcemy przeszukać
    @returns: lista ze słownikiem {page - numer strony, job - stanowisko, html - oryginalny html strony}
    """

    results = []  # zmienna z informacjami o stronach
    page_number = 1

    # będziemy wykonywać pętle tak długo, aż nie trafimy na pustą stronę
    while True:  
        # pobieramy dane dla danego zawodu oraz strony
        bs = download_page(
            job_name=job_name,
            page_number=page_number)
        
        # sprawdzamy czy strona nie jest pusta
        if is_empty_page(bs):
            break
        print(page_number)
        print(job_name)
        print(str(bs))
        print("===============================================")
        
        # pobieramy informacje o ofertach pracy
        results.append({'page': page_number, 'job': job_name, 'html': str(bs)})

        page_number += 1  # przechodzimy do kolejnej strony
        sleep(5)  # czekamy 5 sekund przed kolejnym odpytaniem

    return results

In [17]:
jobs = ['data analyst', 'data scientist', 'data engineer']

In [18]:
browser = Edge()
browser.implicitly_wait(5)

# pobieramy dostępne informacje o zawodach
data = []
for job in jobs:
    data += get_job_offers(job)
browser.close()

In [7]:
dt = datetime.now().strftime("%Y%m%d_%H%M%S")
for d in data:
    with open(
        '../data/raw/{job_name}_{page}_{dt}.html'.format(
            job_name=d['job'], 
            page=d['page'], 
            dt=dt), 
        encoding='UTF-8', 
        mode='w') as f:
        f.write(d['html'])

In [9]:
jobs

['data analyst', 'data scientist', 'data engineer']

In [21]:
browser = Edge()
browser.implicitly_wait(5)

url = 'https://nofluffjobs.com/pl/jobs/?criteria=keyword%3D"data analyst"'
browser.get(url)
load_more_btn = browser.find_element_by_css_selector('button[nfjloadmore]')  # `button` posiadający atrybut `nfjloadmore`
print(load_more_btn)
load_more_btn.click()


AttributeError: 'WebDriver' object has no attribute 'find_element_by_css_selector'